In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats
import db_queries

!whoami
!date

alibow
Mon Jun 22 17:07:14 PDT 2020


In [2]:
output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/25draw_5seed/india/2020_06_19_18_11_00/count_data/']

locations = ['India']

In [ ]:
#NOTE: this was an incomplete run, so comparisons across scenarios are not valid

In [3]:
births = pd.read_hdf(output_dirs[0] + 'births.hdf')
births.head()

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births,21,baseline,99.0
1,2020,female,covered,live_births,21,baseline,2.0
2,2020,female,covered,live_births,21,baseline,20.0
3,2020,female,covered,live_births,21,folic_acid_fortification_scale_up,124.0
4,2020,female,covered,live_births,21,folic_acid_fortification_scale_up,3.0


In [4]:
births_by_scenario = births.groupby(['input_draw','scenario','sex']).sum().groupby(['scenario','sex']).mean()
births_by_scenario

value
scenario                               sex             
baseline                               female  20453.72
                                       male    22470.48
folic_acid_fortification_scale_up      female  20645.68
                                       male    22652.84
iron_folic_acid_fortification_scale_up female  19752.36
                                       male    21675.60
iron_fortification_scale_up            female  20464.68
                                       male    22461.88
vitamin_a_fortification_scale_up       female  20096.92
                                       male    22057.72

In [9]:
ntd_births = ntd_births.groupby(['input_draw','scenario']).sum()

#.loc[ntd_births.value != 0]
ntd_births

value
input_draw scenario                                     
21         baseline                                 37.0
           folic_acid_fortification_scale_up        45.0
           iron_folic_acid_fortification_scale_up   45.0
           iron_fortification_scale_up              29.0
           vitamin_a_fortification_scale_up         37.0
29         baseline                                 44.0
           folic_acid_fortification_scale_up        43.0
           iron_folic_acid_fortification_scale_up   43.0
           iron_fortification_scale_up              44.0
           vitamin_a_fortification_scale_up         44.0
55         baseline                                 27.0
           folic_acid_fortification_scale_up        22.0
           iron_folic_acid_fortification_scale_up   27.0
           iron_fortification_scale_up              27.0
           vitamin_a_fortification_scale_up         27.0
78         baseline                                 22.0
           folic_acid_fortification_scale_up        16.0
           iron_folic_acid_fortification_scale_up   23.0
           iron_fortification_scale_up              25.0
           vitamin_a_fortification_scale_up         29.0
155        baseline                                 37.0
           folic_acid_fortification_scale_up        28.0
           iron_folic_acid_fortification_scale_up   23.0
           iron_fortification_scale_up              37.0
           vitamin_a_fortification_scale_up         37.0
223        baseline                                 40.0
           folic_acid_fortification_scale_up        39.0
           iron_folic_acid_fortification_scale_up   30.0
           iron_fortification_scale_up              40.0
           vitamin_a_fortification_scale_up         33.0
...                                                  ...
650        baseline                                 37.0
           folic_acid_fortification_scale_up        36.0
           iron_folic_acid_fortification_scale_up   36.0
           iron_fortification_scale_up              37.0
           vitamin_a_fortification_scale_up         30.0
674        baseline                                 31.0
           folic_acid_fortification_scale_up        31.0
           iron_folic_acid_fortification_scale_up   27.0
           iron_fortification_scale_up              16.0
           vitamin_a_fortification_scale_up         31.0
680        baseline                                 30.0
           folic_acid_fortification_scale_up        22.0
           iron_folic_acid_fortification_scale_up   28.0
           iron_fortification_scale_up              25.0
           vitamin_a_fortification_scale_up         18.0
733        baseline                                 27.0
           folic_acid_fortification_scale_up        31.0
           iron_folic_acid_fortification_scale_up   28.0
           iron_fortification_scale_up              20.0
           vitamin_a_fortification_scale_up         31.0
829        baseline                                 25.0
           folic_acid_fortification_scale_up        30.0
           iron_folic_acid_fortification_scale_up   24.0
           iron_fortification_scale_up              31.0
           vitamin_a_fortification_scale_up         25.0
946        baseline                                 40.0
           folic_acid_fortification_scale_up        39.0
           iron_folic_acid_fortification_scale_up   39.0
           iron_fortification_scale_up              40.0
           vitamin_a_fortification_scale_up         40.0

[125 rows x 1 columns]

In [6]:
ntd_births_by_scenario = ntd_births.groupby(['input_draw','scenario','sex']).sum().groupby(['scenario','sex']).mean()
ntd_births_by_scenario

value
scenario                               sex          
baseline                               female  15.80
                                       male    15.80
folic_acid_fortification_scale_up      female  15.64
                                       male    15.76
iron_folic_acid_fortification_scale_up female  14.60
                                       male    14.68
iron_fortification_scale_up            female  15.96
                                       male    15.16
vitamin_a_fortification_scale_up       female  16.00
                                       male    15.04

In [7]:
ntd_bp = ntd_births_by_scenario / births_by_scenario
ntd_bp

value
scenario                               sex             
baseline                               female  0.000772
                                       male    0.000703
folic_acid_fortification_scale_up      female  0.000758
                                       male    0.000696
iron_folic_acid_fortification_scale_up female  0.000739
                                       male    0.000677
iron_fortification_scale_up            female  0.000780
                                       male    0.000675
vitamin_a_fortification_scale_up       female  0.000796
                                       male    0.000682

# Check artifact NTD BP

In [8]:
art = Artifact('/share/costeffectiveness/artifacts/vivarium_conic_lsff/india.hdf', 
                   filter_terms=['year_start == 2017', 'age_start < 5'])
data = art.load('cause.neural_tube_defects.birth_prevalence')
data_mean = pd.DataFrame(data.mean(axis=1)).rename(columns={0:'value'})
data_limits = pd.DataFrame(data.quantile([0.025,0.975], axis=1)).transpose().rename(columns={0.025:'lower',
                                                                                             0.975:'upper'})
data = data_mean.merge(data_limits, right_index=True, left_index=True).reset_index()
data.head()

,location,sex,year_start,year_end,value,lower,upper
0,India,Female,2017,2018,0.000762,0.000678,0.000852
1,India,Male,2017,2018,0.000714,0.000636,0.000801
